<a href="https://colab.research.google.com/github/chetannitk/BeerDataScienceAssignment/blob/master/notebooks/02-Beer_review_natural_language_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-success">
<h1> Beer Data Science Assignment </h1>
</div>


In [1]:
from IPython.display import HTML
import pandas as pd
import os
import numpy as np
from datetime import datetime
import wordcloud
import warnings
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
warnings.filterwarnings('ignore')

#BASE_PATH = "../"
#INPUT_DATA_PATH = os.path.join(BASE_PATH, "data/raw/BeerDataScienceProject.csv")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  # Remove the CWD from sys.path while we load stuff.


In [2]:
## Upload BeerDataScienceProject.csv file from local system.
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/data/BeerDataScienceProject.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!ls -lrt /tmp

total 405176
drwx------ 2 root root      4096 May 31 13:46 tmpf9jpugwd
srw------- 1 root root         0 May 31 13:48 drivefs_ipc.0
srw------- 1 root root         0 May 31 13:48 drivefs_ipc.0_shell
drwxr-xr-x 3 root root      4096 May 31 13:58 tfhub_modules
-rw-r--r-- 1 root root 414885425 May 31 14:40 BeerDataScienceProject.csv


In [0]:
import io
df_beer_raw = pd.read_csv('/tmp/BeerDataScienceProject.csv', encoding="ISO-8859-1")

In [5]:
df_beer_raw.shape

(528870, 13)

In [6]:
df_beer_raw.dtypes

beer_ABV              float64
beer_beerId             int64
beer_brewerId           int64
beer_name              object
beer_style             object
review_appearance     float64
review_palette        float64
review_overall        float64
review_taste          float64
review_profileName     object
review_aroma          float64
review_text            object
review_time             int64
dtype: object

### Data Cleaning
```In the below cell we can see that there are some columns where data is missing. 
   We need to remove those data points from our data set.
```

In [7]:
df_beer_raw.isna().sum()

beer_ABV              20280
beer_beerId               0
beer_brewerId             0
beer_name                 0
beer_style                0
review_appearance         0
review_palette            0
review_overall            0
review_taste              0
review_profileName      115
review_aroma              0
review_text             119
review_time               0
dtype: int64

In [0]:
df_beer = df_beer_raw.dropna()

In [9]:
df_beer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508358 entries, 0 to 528796
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   beer_ABV            508358 non-null  float64
 1   beer_beerId         508358 non-null  int64  
 2   beer_brewerId       508358 non-null  int64  
 3   beer_name           508358 non-null  object 
 4   beer_style          508358 non-null  object 
 5   review_appearance   508358 non-null  float64
 6   review_palette      508358 non-null  float64
 7   review_overall      508358 non-null  float64
 8   review_taste        508358 non-null  float64
 9   review_profileName  508358 non-null  object 
 10  review_aroma        508358 non-null  float64
 11  review_text         508358 non-null  object 
 12  review_time         508358 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 54.3+ MB


In [0]:
df_beer = df_beer.reset_index()
df_beer['id'] = df_beer.index

## Data Changes in Raw and Clean dataset

In [11]:
print("Raw data size: ", df_beer_raw.shape[0])
print("Clean data size: ", df_beer.shape[0])
print("Data points difference: ", df_beer_raw.shape[0]-df_beer.shape[0])

Raw data size:  528870
Clean data size:  508358
Data points difference:  20512


### Describe the categorical features

In [12]:
df_beer.describe(include=[np.object])

,beer_name,beer_style,review_profileName,review_text
count,508358,508358,508358,508358
unique,14028,104,22563,508007
top,Sierra Nevada Celebration Ale,American IPA,northyorksammy,#NAME?
freq,2998,42092,1732,90


### Describe the numerical features

In [13]:
df_beer.describe(include=[np.number])

,index,beer_ABV,beer_beerId,beer_brewerId,review_appearance,review_palette,review_overall,review_taste,review_aroma,review_time,id
count,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,508358.000000,5.083580e+05,508358.000000
mean,263669.490377,7.017400,21824.120840,2534.273207,3.872676,3.768993,3.840805,3.775327,3.827646,1.226175e+09,254178.500000
std,152590.091973,2.204528,22124.969107,5237.843825,0.601764,0.682355,0.706408,0.665586,0.715121,7.530843e+07,146750.458409
min,0.000000,0.010000,5.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,8.843904e+08,0.000000
25%,131284.250000,5.300000,1673.000000,132.000000,3.500000,3.500000,3.500000,3.500000,3.500000,1.177202e+09,127089.250000
50%,263756.500000,6.500000,13850.000000,392.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.241503e+09,254178.500000
75%,395689.750000,8.500000,40418.000000,1315.000000,4.000000,4.000000,4.500000,4.000000,4.500000,1.289073e+09,381267.750000
max,528796.000000,57.700000,77310.000000,27980.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.326277e+09,508357.000000


In [0]:
NUMERICAL_ATTRIBUTE = ['beer_ABV', 'review_appearance', 'review_palette', 
                       'review_overall', 'review_taste', 'review_aroma', 'review_time']
CATEGORICAL_ATTRIBUTE = ['beer_name', 'beer_style', 'review_profileName', 'review_text']

In [15]:
df_beer.sample(4)

,index,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time,id
79813,82399,9.2,56761,140,30th Anniversary - Fritz And Ken's Ale,American Double / Imperial Stout,3.5,4.0,4.0,4.0,froghop,4.5,pours a jet black with a medium size brown hea...,1270251733,79813
7686,8141,6.6,3365,395,Hell For Certain,Belgian Dark Ale,4.0,4.0,4.0,3.5,tmitch75,4.5,Clear dark orange (almost red) color. A half i...,1209915737,7686
453894,471948,6.2,36613,392,Muse Farmhouse Ale,Saison / Farmhouse Ale,4.0,3.5,3.0,3.5,weeare138,3.5,Appears a hazy gold with a small white cap tha...,1179328511,453894
379955,394300,7.0,68916,20681,Lunch,American IPA,3.0,3.5,4.0,4.5,kbuzz,4.0,Poured from a 16.9 oz bottle into a snifter. D...,1319762324,379955


<div class="alert alert-info">
Convert the review timestamp to datetime format
</div>

In [0]:
df_beer['d_review_time'] = df_beer.review_time.apply(lambda t: datetime.fromtimestamp(t))

In [0]:
df_beer['d_review_time_year'] = df_beer['d_review_time'].dt.year
df_beer['d_review_time_month'] = df_beer['d_review_time'].dt.month

<div class="alert alert-success">
Q7) How do find similar beer drinkers by using written reviews only?   
</div>

In [18]:
!pip install tensorflow_text

In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
tf.executing_eagerly()

True

In [0]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [22]:
!pip install faiss-gpu 

In [0]:
import faiss

index = faiss.read_index("/content/drive/My Drive/data/beer_review_index_v1")



## REALTIME BEER RECOMENDATION BASED ON REVIEW SIMILARITY.


In [27]:
from  more_itertools import unique_everseen
while True:
    print("Enter review Query? ... Press 'q' to Quit")
    print("=="*10)
    inp_query = input()
    if inp_query in ['q', 'Q']:
        break
    query_vectors = embed([inp_query]).numpy()
    print("Answer")
    distances, indices = index.search(query_vectors, 2)
    for ind in list(unique_everseen(indices[0])):
        print("BEER NAME: ", df_beer[df_beer['id']==ind].beer_name.tolist()[0])
        print("SIMILAR REVIEW TEXT: ", df_beer[df_beer['id']==ind].review_text.tolist()[0])
        print("=="*10)
    print("=="*10)

Enter review Query? ... Press 'q' to Quit
I like a hoppy beer, but I found this a bit unbalanced. Has a great head and smell. Goes great with spaghetti.
Answer
BEER NAME:  Dead Guy Ale
SIMILAR REVIEW TEXT:  I like a hoppy beer, but I found this a bit unbalanced. Has a great head and smell. Goes great with spaghetti.
BEER NAME:  Maudite
SIMILAR REVIEW TEXT:  This beer has everything in my opinion. The appearance gives the initial feeling that this beer might be the best ever. A nice head appears, about 1.5" and the smell provides a plethora of smells, that isn't too hoppy. The alcoholic content is masked very well and doesn't sit too heavy.
Enter review Query? ... Press 'q' to Quit
q
